In [146]:
!pip install librosa
!pip install ipython
!pip install kagglehub

In [147]:
# Audio Preprocessing
import librosa
import librosa.display as dsp
from IPython.display import Audio

# Data Preprocessing
import pandas as pd
import os 
import numpy as np
from sklearn.model_selection import train_test_split

# Data Visualization
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm

In [148]:
import kagglehub

# download dataset
path = kagglehub.dataset_download("sripaadsrinivasan/audio-mnist")

print("Path to dataset files:", path)

Path to dataset files: /home/ec2-user/.cache/kagglehub/datasets/sripaadsrinivasan/audio-mnist/versions/1


In [305]:
def extract_features(file):

    audio,sample_rate = librosa.load(file)

    extracted_features = librosa.feature.mfcc(y=audio,
                                              sr=sample_rate,
                                              n_mfcc=40)
    

    extracted_features = np.mean(extracted_features.T,axis=0)

    return extracted_features


In [316]:
def create_dataset():
    dataset = []
    final_path = path + "/data/"
    for folder in tqdm(range(1,61), colour = 'green'): 
        if folder < 10:
            folder = final_path + "0" + str(folder) + "/"
        else:
            folder = final_path + str(folder) + "/"
        for file in tqdm(os.listdir(folder), colour = 'blue'):
            abs_path = folder + file
            features = extract(abs_path)
            category = file[0]
            dataset.append([features, category])
    return pd.DataFrame(dataset, columns = ['features', 'class'])

In [161]:
df = create_dataset()

100%|██████████| 60/60 [05:42<00:00,  5.72s/it]


In [162]:
df2 = df.sample(frac=1).reset_index(drop=True)
print(df2)

                                                features class
0      [-661.35535, 129.85709, 33.002457, 19.596064, ...     1
1      [-652.4539, 57.61384, 38.444252, 51.153107, 29...     2
2      [-593.8398, -10.590129, 36.06963, 24.341312, 1...     6
3      [-631.115, 57.74864, 37.95098, 43.70374, 21.12...     8
4      [-623.5145, 54.55371, 15.75465, 53.253048, 26....     8
...                                                  ...   ...
29995  [-593.78357, 100.70353, 27.592667, 31.757032, ...     4
29996  [-683.5668, 109.23907, 31.491138, 35.497776, 2...     2
29997  [-597.24884, 135.7962, -4.5543633, 33.63186, 2...     9
29998  [-624.9965, 74.23788, 26.951681, 16.010178, 4....     7
29999  [-600.5901, 97.77441, -4.0980315, 62.084587, 3...     8

[30000 rows x 2 columns]


In [163]:
X = np.array(df2['features'].to_list())
Y = np.array(df2['class'].to_list())

trainX, testX, trainY, testY = train_test_split(X,Y, train_size = 0.75, shuffle=True, random_state=8)

In [165]:
def save_to_parquet(trainX, trainY, testX, testY):
    base_dir = 'data/'
    trainX_path = base_dir + 'trainX.parquet'
    trainY_path = base_dir + 'trainY.parquet'
    testX_path = base_dir + 'testX.parquet'
    testY_path = base_dir + 'testY.parquet'
    
    trainX_df = pd.DataFrame(trainX.reshape(trainX.shape[0], - 1))
    trainX_df.columns = [str(i) for i in trainX_df.columns]
    trainY_df = pd.DataFrame(trainY, columns=['label'])
    testX_df = pd.DataFrame(testX.reshape(testX.shape[0], - 1))
    testX_df.columns = [str(i) for i in testX_df.columns]
    testY_df = pd.DataFrame(testY, columns=['label'])
    
    trainX_df.to_parquet(trainX_path, index = False)
    trainY_df.to_parquet(trainY_path, index = False)
    testX_df.to_parquet(testX_path, index = False)
    testY_df.to_parquet(testY_path, index = False)

In [166]:
save_to_parquet(trainX, trainY, testX, testY)

# Move Data into S3

In [167]:
import boto3

bucket_name = 'sagemaker-final-project-audio'
s3 = boto3.client('s3')
s3.upload_file('data/trainX.parquet', bucket_name, 'train/trainX.parquet')
s3.upload_file('data/trainY.parquet', bucket_name, 'train/trainY.parquet')
s3.upload_file('data/testX.parquet', bucket_name, 'test/testX.parquet')
s3.upload_file('data/testY.parquet', bucket_name, 'test/testY.parquet')


# Train Model


In [168]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

# dataset uri
trainx_data_uri = 's3://sagemaker-final-project-audio/train/trainX.parquet'
trainy_data_uri = 's3://sagemaker-final-project-audio/train/trainY.parquet'
testx_data_uri = 's3://sagemaker-final-project-audio/test/testX.parquet'
testy_data_uri = 's3://sagemaker-final-project-audio/test/testY.parquet'

In [169]:
session = sagemaker.Session()
role = get_execution_role()

In [205]:
hyperparameters = {
    "epochs": 100,
    "learning-rate": 0.01,
    "per-device-train-batch-size": 32,
    "per-device-test-batch-size": 32,
}

In [214]:
estimator = TensorFlow(
    py_version = "py37",
    entry_point="train.py",
    framework_version="2.2",
    hyperparameters=hyperparameters,
    instance_type="ml.p2.xlarge",
    instance_count=1,
    role=role,
    debugger_hook_config=False,
    disable_profiler=True,
    dependencies=['requirements.txt'],
)

In [208]:
!pip install --upgrade sagemaker


  Using cached sagemaker-2.236.0-py3-none-any.whl.metadata (16 kB)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached omegaconf-2.2.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached sagemaker_core-1.0.17-py3-none-any.whl.metadata (4.9 kB)
  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [43 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-g5v0i973/antlr4_python3_runtime.egg-info
      writing /tmp/pip-pip-egg-info-g5v0i973/antlr4_python3_runtime.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-g5v0i973/antlr4_python3_runtime.egg-info/dependency_links.txt
      writing requirements to /tmp/pip-pip-egg-info-g5v0i973/antlr4_python3_runtime.egg-info/requires.txt
      writin

In [215]:
estimator.fit({
    "trainx":trainx_data_uri,
    "trainy":trainy_data_uri,
    "testx":testx_data_uri,
    "testy":testy_data_uri,
})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-12-05-03-15-22-472


2024-12-05 03:15:25 Starting - Starting the training job...
2024-12-05 03:15:50 Starting - Preparing the instances for training......
2024-12-05 03:16:56 Downloading - Downloading the training image...............
2024-12-05 03:19:03 Training - Training image download completed. Training in progress..2024-12-05 03:19:30.422553: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2024-12-05 03:19:30.422763: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-12-05 03:19:30.452511: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2024-12-05 03:19:31,951 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-12-05 03:19:32,256 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:

# Deploy Model

In [218]:
predictor = estimator.deploy(initial_instance_count = 1, instance_type="ml.t2.medium")

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: tensorflow-training-2024-12-05-04-21-04-584
INFO:sagemaker:Creating endpoint-config with name tensorflow-training-2024-12-05-04-21-04-584
INFO:sagemaker:Creating endpoint with name tensorflow-training-2024-12-05-04-21-04-584


----!

In [324]:
np.argmax(predictor.predict(data_point)['predictions'][0])

0

In [325]:
predictor.predict(data_point)

{'predictions': [[0.957491,
   4.43188021e-11,
   3.58249259e-12,
   2.94288197e-17,
   9.50203e-08,
   2.36710093e-07,
   2.67926365e-21,
   1.71509582e-05,
   1.58828561e-26,
   0.0424915217]]}